In [1]:
# !pip install numpy    

In [2]:
# !pip install opencv-python

In [3]:
# !pip install cmake

In [4]:
# !python -m pip install dlib-19.24.1-cp311-cp311-win_amd64.whl

In [5]:
# !pip install dlib

In [6]:
# !pip install pyttsx3

In [7]:
# !pip install scipy

In [8]:
import cv2
import dlib
import pyttsx3
from scipy.spatial import distance

In [24]:
# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Attempt to open the default camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Load the dlib face detector and the 68-point landmark predictor
face_detector = dlib.get_frontal_face_detector()
dlib_facelandmark = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Function to calculate the Eye Aspect Ratio (EAR)
def Detect_Eye(eye):
    # Calculate the vertical distances between points
    poi_A = distance.euclidean(eye[1], eye[5])
    poi_B = distance.euclidean(eye[2], eye[4])
    # Calculate the horizontal distance between points
    poi_C = distance.euclidean(eye[0], eye[3])
    # Compute the EAR
    aspect_ratio_Eye = (poi_A + poi_B) / (2 * poi_C)
    return aspect_ratio_Eye

# Continuously capture frames from the camera
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    # Convert the frame to grayscale for processing
    gray_scale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_detector(gray_scale)

    # Iterate over each detected face
    for face in faces:
        # Get the facial landmarks for the detected face
        face_landmarks = dlib_facelandmark(gray_scale, face)

        leftEye = []  # Coordinates for the left eye
        rightEye = []  # Coordinates for the right eye

        # Extract right eye landmarks and draw lines
        for n in range(42, 48):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            rightEye.append((x, y))
            next_point = n + 1 if n != 47 else 42
            x2 = face_landmarks.part(next_point).x
            y2 = face_landmarks.part(next_point).y
            cv2.line(frame, (x, y), (x2, y2), (0, 255, 0), 1)

        # Extract left eye landmarks and draw lines
        for n in range(36, 42):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            leftEye.append((x, y))
            next_point = n + 1 if n != 41 else 36
            x2 = face_landmarks.part(next_point).x
            y2 = face_landmarks.part(next_point).y
            cv2.line(frame, (x, y), (x2, y2), (255, 255, 0), 1)

        # Calculate the average Eye Aspect Ratio (EAR)
        right_Eye = Detect_Eye(rightEye)
        left_Eye = Detect_Eye(leftEye)
        Eye_Rat = round((left_Eye + right_Eye) / 2, 2)

        # Check if EAR indicates drowsiness
        if Eye_Rat < 0.15:
            cv2.putText(frame, "DROWSINESS DETECTED", (50, 100), cv2.FONT_HERSHEY_PLAIN, 2, (21, 56, 210), 3)
            cv2.putText(frame, "Alert!!!! WAKE UP DUDE", (50, 450), cv2.FONT_HERSHEY_PLAIN, 2, (21, 56, 212), 3)
            engine.say("Alert!!!! WAKE UP")
            engine.runAndWait()

    # Display the frame with annotations
    cv2.imshow("Drowsiness Detector", frame)

    # Exit on pressing 'q'
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()